# Attention is all you need

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
import sys
from typing import Callable
seaborn.set_context(context="talk")
%matplotlib inline

%load_ext autoreload
%autoreload 2
sys.path.append('../')
from lib.logger.logging import LoggerInitializer
import logging

LoggerInitializer().init()
logger = logging.getLogger('development')

[2024-04-01 18:51:34] 38532 root {logging-48} INFO - Current process ID: 38532


# Model Archetecture
## Encoder-Decoder
* the encoder maps an input sequence of symbol representations (x1,...,xn) to a sequence of continuous representations z = (z1,...,zn). Given z, the decoder then generates an output sequence (y1, ..., ym) of symbols one element at a time.
```
    X := x1, x2, x3, x4, ......, xn  (represent featuring of the symbol/word)
    Z := z1, z2, z3, z4, ......, zn  (represent order/sequence)
         ↓   ↓    ↓   ↓           ↓
    Y := y1, y2, y3, y4, ......, yn  (represnt the output gievn zi)
```
<!-- Image(filename='../references/Attention/overall_structure.jpeg') -->
![alt text](../references/Attention/overall_structure.jpeg "Title")

In [8]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture, base for this and many other model
    """
    def __init__(
        self,
        encoder: nn.Module,
        decoder: nn.Module,
        src_embed: np.ndarray, 
        tgt_embed: np.ndarray,
        generator: nn.Module
    ):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed # x
        self.tgt_embed = tgt_embed # z
        self.genrator = generator  # y = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        """Take in and process: masked src (x) and target sequence
        """
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [9]:
class Generator(nn.Module):
    """ Define standard linear + softmax generation step

    Args:
        nn (_type_): _description_
    """
    def __init__(self, d_model, vocal) -> None:
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocal)
    
    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=1)

# Encoder

* N = 6 

![alt text](../references/Attention/encoder.png "Title")

In [10]:
def clones(module, N):
    """ Produce N identical Layers

    Args:
        module (_type_): _description_
        N (_type_): _description_
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

### The encoder is composed of a stack of N = 6 identical layers.
### We employ a residual connection [11] around each of the two sub-layers, followed by layer normalization [1]. 


In [11]:
class LayerNorm(nn.Module):
    def __init__(self, no_of_features, eps=1e-6) -> None:
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(no_of_features)) 
        self.b_2 = nn.Parameter(torch.ones(no_of_features)) 
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [12]:
class Encoder(nn.Module):
    """Core encoder is a stack of N layers

    Args:
        nn (_type_): _description_
    """
    def __init__(self, layer, N) -> None:
        super(Encoder, self).__init__()
        self.layers = clones(layer, N) # N x Layer
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, mask):
        """Pass the input and mask through each layer in turn 

        Args:
            x (_type_): _description_
            mask (_type_): _description_
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x) # followed by layer normalization

### We employ a residual connection [11] around each of the two sub-layers, followed by layer normalization [1]. 

### That is, the output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself.
### To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension dmodel = 512.

In [13]:
class SublayerConnection(nn.Module):
    """a residual connection followed by a layer norm
    
    Args:
        nn (_type_): _description_
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        """_summary_

        Args:
            x (_type_): _description_
            sublayer (_type_): here should be fully connected feedforward or multi-head self-attention
        """
        return x + self.dropout(sublayer(self.norm(x))) # x + : residual connection
        

### Each layer has two sub-layers. 
        * The first is a multi-head self-attention mechanism, 
        * and the second is a simple, positionwise fully connected feed-forward network.

In [14]:
class EncoderLayer(nn.Module):
    """_summary_

    Args:
        nn (_type_): _description_
    """
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2) # 2 sublayer connections
         
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x,  mask)) # first sublayer will be self attention multi head model
        return self.sublayer[1](x, self.feed_forward) # the second sublayer will be fully connected feed-forward model


# Decoder

* N = 6 

![alt text](../references/Attention/decoder.png "Title")

In [ ]:
class 